In [1]:
from cvxpy import *
import numpy as np
import pandas as pd
import random
random.seed(58)
from dataset import extract_matrix_users_movies_ratings

In [2]:
data = pd.read_csv('./data/data_train.csv')
number_of_users, number_of_movies = (10000, 1000)
matrix, _ = extract_matrix_users_movies_ratings(data)

In [3]:
matrix.shape

(10000, 1000)

In [4]:
matrix2 = matrix[:100]

In [17]:
def learnMMMF(y, c):
    n,m = y.shape
    n_obs = np.count_nonzero(y)
    obs = np.nonzero(y)

    A = Variable((n,n))
    B = Variable((m,m))
    X = Variable((n,m))

    t = Variable()
    e = Variable((n_obs,))

    objective = Minimize(t + c * sum(e))

    constraints = []
    constraints.append(bmat([[A,X],[X.T,B]]) >> 0)
    constraints.append(diag(A) <= t)
    constraints.append(diag(B) <= t)
    constraints.append(e >= 0)

    for x in enumerate(zip(obs[0], obs[1])):
        ind = x[0]
        i, a = x[1][0], x[1][1]
        constraints.append(y[i,a] * X[i,a] >= 1 - e[ind])

    prob = Problem(objective, constraints)
    result = prob.solve(verbose=True, max_iters=250)
    return result

In [25]:
a = learnMMMF(matrix2, 1.0)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jul 17 02:21:00 PM: Your problem has 1121784 variables, 11787 constraints, and 0 parameters.
(CVXPY) Jul 17 02:21:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 17 02:21:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 17 02:21:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 17 02:21:02 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 17 02:21:02 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStu

In [28]:
prob

Problem(Minimize(Expression(AFFINE, UNKNOWN, ())), [PSD(Expression(AFFINE, UNKNOWN, (3, 3))), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONPOSITIVE, ())), Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))])

In [14]:
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    trace(A[i] @ X) == b[i] for i in range(p)
]
prob = Problem(Minimize(trace(C @ X)),
                  constraints)
prob.solve(kktsolver=ROBUST_KKTSOLVER, verbose=True)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jul 17 11:38:01 AM: Your problem has 9 variables, 4 constraints, and 0 parameters.
(CVXPY) Jul 17 11:38:01 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 17 11:38:01 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 17 11:38:01 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 17 11:38:01 AM: Compiling problem (target solver=SCS).
(CVXPY) Jul 17 11:38:01 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> S

ValueError: Error parsing inputs


In [ ]:
learnMMMF(matrix, 1.0)